In [1]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import numpy as np
import import_ipynb
import misc_functions as msc_fct

importing Jupyter notebook from misc_functions.ipynb


In [2]:
def QVI_solver(t, dt, kappa, xi, phi, q, q_target, lamb):
    '''
    Solves the quasi variational inequality using a forward euler timestepping appraoch
    
    imputs:
    kappa: higher kappa lower depth that market orders will reach on average
    xi: half spread
    phi: parameter effecting how strongly agent penalised for deviating from target liquidation schedule
    lamb: arrival intensity of market orders
    t: array of times
    dt: incremental increase in time
    q: range of quantities of inventory
    q_target: target inventory schedule over time
    
    returns:
    omega: matrix of solutions to the HJB
    exe: boolian matrix with true value if market order is executed
    '''
    
    Ndt = t.shape[0]
    Nq = q.shape[0]
    
    # Setting up matrix of solutions
    omega = np.full((Nq, Ndt), np.NaN)
    
    # Terminal conditions for all q
    omega[:, omega.shape[1]-1] = np.exp(-kappa * xi * q) #indexing starts at zero not 1

    # Boundary conditions along q = 0
    omega[0, :] = np.exp(-kappa * phi * (np.sum(np.power(q_target, 2) * dt) - np.cumsum(np.power(q_target, 2) * dt))) 
    #The target schedule integral is computed via trapezodial approximation, summing each element in the 
    #array multipled by the incremental increase in time subracted by the sum multiplied by the incremental 
    #increase in time up to a given point t to insure the integral is given between t and T
    
    # Matrix to show if market order is executed
    exe = np.zeros((Nq, Ndt))
    exe[:, exe.shape[1]-1] = 1
    
    for k in range(Ndt - 2, -1, -1): 
        # Range Ndt to -1 decreasing in increments of 1 each time Ndt -2 to correct for index starting at 0 & 
        # having the terminal condition. Note work backwards from terminal condition.
        
        # Solve the HJB in the continuation region from t+dt to t
        omega[1:omega.shape[0], k] = omega[1:omega.shape[0], k+1] + \
                                     dt * (-kappa * phi * np.multiply(np.power((q[1:q.shape[0]] - q_target[k+1]), 2), omega[1:omega.shape[0], k+1]) +
                                           np.exp(-1) * lamb * omega[0:(omega.shape[0]-1), k+1])
        # Use of forward euler scheme

        idx_exe = np.insert(np.exp(-kappa * xi) * omega[0:(omega.shape[0]-1), k] > omega[1:omega.shape[0], k], 0, 0)
        # checks to see if a market order has been executed while ensuring that a market order cannot be executed when
        # there is no inventory
        exe[:, k] = idx_exe
        
        # checks to see if market order is executed or limit order is used
        omega[1:omega.shape[0], k] = np.maximum(np.exp(-kappa * xi) * omega[0:(omega.shape[0]-1), k], omega[1:omega.shape[0], k])
    return omega, exe

In [3]:
def find_opt_t(exe, t):
    '''
    Solves for the optimal time to execute a market order for each inventory level
    
    imputs:
    exe: booloan matrix with execution marked as true
    t: vector of time elapsed
    
    output:
    t_opt: optimal time to execute market orders
    
    '''
    
    # number of units of inventory
    Nq = exe.shape[0]
    
    # optimal time to execute market order for each inventory
    t_opt = np.full(Nq, np.NaN)
    
    for k in range(0, Nq, 1):
        # gives the index where the market orders are executed
        idx = np.where(exe[k, :] == 1)[0]
        
        # sees if a market order is executed
        if idx.shape[0] > 0:
            
            # gives the times that a market order is executed
            t_opt[k] = t[idx[0]]
    return t_opt

In [4]:
def find_delta(kappa, omega, Nq, Ndt):
    '''
    Finds the optimal delta to execute a limit order
    
    imputs:
    kappa: higher kappa lower depth that market orders will reach on average
    omega: matrix output from QVI_solver
    Nq: number of units of inventory
    Ndt: number of timesteps
    
    outputs:
    delta: optimal delta for each unit of inventory and time step
    '''
    delta = np.full((Nq + 1, Ndt + 1), np.NaN)
    delta[1:delta.shape[0], :] = (1 / kappa) + (1 / kappa) * np.log(np.divide(omega[1:omega.shape[0], :], omega[0:(omega.shape[0]-1), :]))
    return delta

In [5]:
def generate_simulations(Nsims, s0, Ndt, Nq, dt, delta, lamb, kappa, sigma, xi, t_opt, t, omega):
    '''
    Computes simulations of the liquidation process under a given policy
    
    imputs:
    Nsim: the number of simulation
    S0: the inital midprice
    Ndt: the number of timesteps
    Nq: the inital amount of inventory
    dt: the time step size
    delta: the matrix of depths
    lamb: arrival intensity of market orders
    kappa: higher kappa lower depth that market orders will reach on average
    sigma: varaition of the midprice
    xi: the half spread
    t: array of times
    omega: matrix of outputs from solving HJB
    
    outputs:
    deltaPath: delta for each simulation over time
    Qpath: inventory path for each simulation over time
    isMO: idicates where a market oder is executed for each simulation
    Xpath: path of wealth process for each simulation
    Spath: path of mid-price for each simulation
    pricePerShare: average price per share of liquidated inventory
    twap: time weighted average price
    V: the value function for each simulation over time
    '''
    
    
    Qpath = np.full((Nsims, Ndt + 1), np.NaN)
    # Set starting inventory
    Qpath[:, 0] = Nq
    
    Xpath = np.full((Nsims, Ndt + 1), np.NaN)
    # Set starting cash
    Xpath[:, 0] = 0
    
    Spath = np.full((Nsims, Ndt + 1), np.NaN)
    # Set starting Mid-price
    Spath[:, 0] = s0
    
    # Preconditioning matricies
    deltaPath = np.full((Nsims, Ndt + 1), np.NaN)
    isFilled = np.full((Nsims, Ndt + 1), np.NaN)
    isMO = np.zeros((Nsims, Ndt + 1))
    pricePerShare = np.full((Nsims, Ndt + 1), np.NaN)

    mu = 0
    for k in range(0, Ndt, 1):
        
        idx = Qpath[:, k] > 0 
        # Checks to see if the current inventory is zero for all the simulations, the output of this is a boolian array

        deltaPath[idx, k] = delta[Qpath[idx, k].astype(int), k] # Uses fancy indexing, for each simulation that does not
        # have zero inventory the delta for that simulation is extracted given the time and inventory
        
        isFilled[idx, k] = np.random.rand(np.sum(idx)) < msc_fct.nan_overwrite(lamb * np.exp(-kappa * deltaPath[idx, k]) * dt, np.Inf) 
        # changing the nan values to infinity, lam*dt - prob of market order arriving, np.exp(-kappa * deltaPath[idx, k]) - prob
        # of depth being hit conditional on market order arriving

        deltaPath[(1 - idx).astype(bool), k] = np.NaN 
        isFilled[(1 - idx).astype(bool), k] = 0
        # updating the simulation in deltaPath and isFilled that had zero inventory

        idx = np.logical_and((k + 1) * dt > t_opt[Qpath[:, k].astype(int)], Qpath[:, k] > 0) 
        # creates an index which has 1 if it is optimal to execute a market order and if there is still inventory left
        # for the given simulation
        isMO[idx, k] = 1 # for the given simulation if a market order is executed outputs 1
        
        isFilled[:, k] = np.logical_and(1 - isMO[:, k], isFilled[:, k]) # case where a market order is not issued by the agent
        # and the limit order is filled i.e. depth is reached
        
        idx = Qpath[:, k] > 0 #checks to see if the current inventory is zero for all the simulations, the output of this is a
        # boolian array
        
        Xpath[idx, k + 1] = Xpath[idx, k] + np.multiply(isFilled[idx, k], Spath[idx, k] + deltaPath[idx, k]) + \
                            np.multiply(isMO[idx, k], Spath[idx, k] - xi)
        # for the case that inventory remains the wealth process is updated such that if a limit order or market order has 
        # been executed the wealth process increases acordingly 
        
        
        Xpath[(1 - idx).astype(bool), k + 1] = Xpath[(1 - idx).astype(bool), k] # keeps the wealth process the same if there 
        # was no inventory left
        
        Qpath[:, k + 1] = Qpath[:, k] - isMO[:, k] - isFilled[:, k] #updates the path of the inventory depending on if a market
        # order or limit order is executed
        
        Spath[:, k + 1] = Spath[:, k] + mu * dt + sigma * np.sqrt(dt) * np.random.randn(Nsims) #updates the path of the midprice

        idx = np.logical_and(Qpath[:, k + 1] < Nq, Qpath[:, k + 1] > 0) # boolian index with true value if the quantity of inventory
        # at the current time step is less than the starting quantity but also greater than zero

        pricePerShare[idx, k + 1] = np.divide(Xpath[idx, k + 1], Nq - Qpath[idx, k + 1]) # takes the wealth at the current time
        # step and averages it over the amount of inventory sold

        idx = Qpath[:, k + 1] == 0 # simulation where there is no inventory left
        pricePerShare[idx, k + 1] = pricePerShare[idx, k] # if no inventory left then price per share will not change over time

        
    # no longer in the for loop
    Xpath[:, Xpath.shape[1] - 1] += np.multiply(Qpath[:, Qpath.shape[1] - 1], Spath[:, Spath.shape[1] - 1] - xi)
    # increase in the wealth process at the terminal time

    idx = Qpath[:, Qpath.shape[1] - 1] > 0 # index that will return values of 1 if the final value of the inventory
    # is not zero
    
    Qpath[:, Qpath.shape[1] - 1] = 0
    # Quantity of inventory decreasing to 0 at the final time for all simulations as market orders issued
    
    pricePerShare[idx, pricePerShare.shape[1] - 1] = np.divide(Xpath[idx, Xpath.shape[1] - 1],
                                                              Nq - Qpath[idx, Qpath.shape[1] - 1]) 
    # computes the final price per share after liquidation of final inventory

    twap = np.divide(np.cumsum(Spath[:, 0:(Spath.shape[1] - 1)] * dt, axis=1), t[1:t.shape[0]]) - xi
    # gives the value of liquidating a single stock at TWAP across the entire range of times hence - xi
    
    twap = np.concatenate((Spath[:, 0][:, np.newaxis], twap), axis=1)
    # adding a new axis which contains the twap for each simulation of the underlying
    
    # Value function
    V = np.full((Nsims, Ndt + 1), np.NaN)
    
    h = (1/kappa)*np.log(omega)
    # Transforming omega
    
    # Computing the value function
    for k in range(0, Nsims, 1):
        for n in range(0, Ndt, 1):
            V[k,n] = Xpath[k,n] + Qpath[k,n]*Spath[k,n] + h[Qpath[k,n].astype(int),n]
            
    V[:,-1] = Xpath[:,-1]
        
    return deltaPath, Qpath, isMO, Xpath, Spath, pricePerShare, twap, V

In [ ]:
def QVI_solver_percent(t, dt, kappa, xi, phi, q, q_target, lamb, eta):
    '''
    Solves the quasi variational inequality using a forward euler timestepping appraoch when liquidating percentage of porfolio
    
    imputs:
    kappa: higher kappa lower depth that market orders will reach on average
    xi: half spread
    phi: parameter effecting how strongly agent penalised for deviating from target liquidation schedule
    lamb: arrival intensity of market orders
    t: array of times
    dt: incremental increase in time
    q: range of quantities of inventory
    q_target: target inventory schedule over time
    
    returns:
    omega: matrix of solutions to the HJB
    exe: boolian matrix with true value if market order is executed
    '''
    
    Ndt = t.shape[0]
    Nq = q.shape[0]
    
    # Setting up matrix of solutions
    omega = np.full((Nq, Ndt), np.NaN)
    
    # Terminal conditions for all q
    omega[:, omega.shape[1]-1] = np.exp(-kappa * xi) #indexing starts at zero not 1

    # Boundary conditions along q = 0
    omega[0, :] = 1 
    #The target schedule integral is computed via trapezodial approximation, summing each element in the 
    #array multipled by the incremental increase in time subracted by the sum multiplied by the incremental 
    #increase in time up to a given point t to insure the integral is given between t and T
    
    # Matrix to show if market order is executed
    exe = np.zeros((Nq, Ndt))
    exe[:, exe.shape[1]-1] = 1
    
    for k in range(Ndt - 2, -1, -1): 
        # Range Ndt to -1 decreasing in increments of 1 each time Ndt -2 to correct for index starting at 0 & 
        # having the terminal condition. Note work backwards from terminal condition.
        
        # Solve the HJB in the continuation region from t+dt to t
        omega[1:omega.shape[0], k] = omega[1:omega.shape[0], k+1] + \
                                     dt * (np.exp(-1) * lamb * omega[0:(omega.shape[0]-1), k+1])
        # Use of forward euler scheme

        idx_exe = np.insert(np.exp(-kappa * xi/eta) * omega[0:(omega.shape[0]-1), k] > omega[1:omega.shape[0], k], 0, 0)
        # checks to see if a market order has been executed while ensuring that a market order cannot be executed when
        # there is no inventory
        exe[:, k] = idx_exe
        
        # checks to see if market order is executed or limit order is used
        omega[1:omega.shape[0], k] = np.maximum(np.exp(-kappa * xi) * omega[0:(omega.shape[0]-1), k], omega[1:omega.shape[0], k])
    return omega, exe